# FedNAMs+ Baseline Comparison

This notebook compares FedNAMs+ against baseline methods:
- FedAvg CNN (standard federated learning)
- FedAvg + Grad-CAM (post-hoc explanations)
- Centralized NAM (privacy baseline)

## Comparison Dimensions
1. Classification performance
2. Explanation quality
3. Uncertainty quantification
4. Communication efficiency
5. Privacy guarantees

## 1. Setup

In [ ]:
import sys
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Setup plotting
sns.set_style('whitegrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

## 2. Load Results from All Methods

In [ ]:
# Define experiment directories
experiments = {
    'FedNAMs+': 'outputs/fednams_plus_baseline',
    'FedAvg CNN': 'outputs/fedavg_cnn_baseline',
    'FedAvg + GradCAM': 'outputs/fedavg_gradcam_baseline',
    'Centralized NAM': 'outputs/centralized_nam_baseline'
}

# Load all results
all_results = {}
for name, path in experiments.items():
    results_path = Path(path) / 'results.json'
    if results_path.exists():
        with open(results_path, 'r') as f:
            all_results[name] = json.load(f)
        print(f"✓ Loaded: {name}")
    else:
        print(f"✗ Not found: {name} ({results_path})")

print(f"\nLoaded {len(all_results)} experiments")

## 3. Classification Performance Comparison

In [ ]:
# Extract classification metrics
metrics_data = []
for name, results in all_results.items():
    test_metrics = results.get('test_metrics', {})
    metrics_data.append({
        'Method': name,
        'Accuracy': test_metrics.get('accuracy', 0),
        'F1-Score': test_metrics.get('f1', 0),
        'AUC-ROC': test_metrics.get('auc_roc', 0),
        'AUC-PR': test_metrics.get('auc_pr', 0)
    })

metrics_df = pd.DataFrame(metrics_data)

print("\n=== Classification Performance Comparison ===")
display(metrics_df.style.format({
    'Accuracy': '{:.4f}',
    'F1-Score': '{:.4f}',
    'AUC-ROC': '{:.4f}',
    'AUC-PR': '{:.4f}'
}).background_gradient(cmap='YlGn', subset=['Accuracy', 'F1-Score', 'AUC-ROC', 'AUC-PR']))

In [ ]:
# Visualize classification metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
metrics_to_plot = ['Accuracy', 'F1-Score', 'AUC-ROC', 'AUC-PR']
colors = sns.color_palette('Set2', len(metrics_df))

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    bars = ax.bar(metrics_df['Method'], metrics_df[metric], color=colors)
    ax.set_ylabel('Score')
    ax.set_title(f'{metric} Comparison', fontsize=12, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height + 0.01,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('outputs/classification_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Explanation Quality Comparison

In [ ]:
# Extract explanation metrics
explanation_data = []
for name, results in all_results.items():
    exp_metrics = results.get('explanation_metrics', {})
    if exp_metrics:  # Only include if explanation metrics exist
        explanation_data.append({
            'Method': name,
            'SHAP Consistency': exp_metrics.get('shap_consistency', 0),
            'Feature Stability': exp_metrics.get('feature_stability', 0),
            'Cross-Client Agreement': exp_metrics.get('cross_client_agreement', 0)
        })

if explanation_data:
    explanation_df = pd.DataFrame(explanation_data)
    
    print("\n=== Explanation Quality Comparison ===")
    display(explanation_df.style.format({
        'SHAP Consistency': '{:.4f}',
        'Feature Stability': '{:.4f}',
        'Cross-Client Agreement': '{:.4f}'
    }).background_gradient(cmap='YlGn', subset=['SHAP Consistency', 'Feature Stability', 'Cross-Client Agreement']))
    
    # Visualize
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(explanation_df))
    width = 0.25
    
    ax.bar(x - width, explanation_df['SHAP Consistency'], width, 
           label='SHAP Consistency', alpha=0.8)
    ax.bar(x, explanation_df['Feature Stability'], width, 
           label='Feature Stability', alpha=0.8)
    ax.bar(x + width, explanation_df['Cross-Client Agreement'], width, 
           label='Cross-Client Agreement', alpha=0.8)
    
    ax.set_ylabel('Score')
    ax.set_title('Explanation Quality Metrics Comparison', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(explanation_df['Method'], rotation=45, ha='right')
    ax.legend()
    ax.set_ylim(0, 1)
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('outputs/explanation_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No explanation metrics available for comparison")

## 5. Uncertainty Quantification Comparison

In [ ]:
# Extract uncertainty metrics
uncertainty_data = []
for name, results in all_results.items():
    unc_metrics = results.get('uncertainty_metrics', {})
    if unc_metrics:  # Only include if uncertainty metrics exist
        uncertainty_data.append({
            'Method': name,
            'Coverage': unc_metrics.get('coverage', 0),
            'Avg Set Size': unc_metrics.get('avg_set_size', 0),
            'Target': unc_metrics.get('target_confidence', 0.9)
        })

if uncertainty_data:
    uncertainty_df = pd.DataFrame(uncertainty_data)
    
    print("\n=== Uncertainty Quantification Comparison ===")
    display(uncertainty_df.style.format({
        'Coverage': '{:.4f}',
        'Avg Set Size': '{:.2f}',
        'Target': '{:.2f}'
    }))
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Coverage comparison
    x = np.arange(len(uncertainty_df))
    width = 0.35
    axes[0].bar(x - width/2, uncertainty_df['Target'], width, 
                label='Target', alpha=0.7, color='gray')
    axes[0].bar(x + width/2, uncertainty_df['Coverage'], width, 
                label='Achieved', alpha=0.7, color='green')
    axes[0].set_ylabel('Coverage')
    axes[0].set_title('Coverage: Target vs Achieved', fontsize=12, fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(uncertainty_df['Method'], rotation=45, ha='right')
    axes[0].legend()
    axes[0].set_ylim(0, 1)
    axes[0].grid(axis='y', alpha=0.3)
    
    # Set size comparison
    bars = axes[1].bar(uncertainty_df['Method'], uncertainty_df['Avg Set Size'], 
                       color=sns.color_palette('Set2', len(uncertainty_df)))
    axes[1].set_ylabel('Average Set Size')
    axes[1].set_title('Prediction Set Size Comparison', fontsize=12, fontweight='bold')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2, height + 0.05,
                    f'{height:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('outputs/uncertainty_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No uncertainty metrics available for comparison")

## 6. Communication Cost Comparison

In [ ]:
# Extract communication metrics (only for federated methods)
comm_data = []
for name, results in all_results.items():
    if 'Centralized' not in name:  # Skip centralized methods
        comm_metrics = results.get('communication_metrics', {})
        if comm_metrics:
            comm_data.append({
                'Method': name,
                'MB per Round': comm_metrics.get('mb_per_round', 0),
                'Total MB': comm_metrics.get('total_mb', 0),
                'Rounds': comm_metrics.get('total_rounds', 0)
            })

if comm_data:
    comm_df = pd.DataFrame(comm_data)
    
    print("\n=== Communication Cost Comparison (Federated Methods) ===")
    display(comm_df.style.format({
        'MB per Round': '{:.2f}',
        'Total MB': '{:.2f}',
        'Rounds': '{:d}'
    }))
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Per-round cost
    bars = axes[0].bar(comm_df['Method'], comm_df['MB per Round'], 
                       color=sns.color_palette('Set2', len(comm_df)))
    axes[0].set_ylabel('MB per Round')
    axes[0].set_title('Communication Cost per Round', fontsize=12, fontweight='bold')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(axis='y', alpha=0.3)
    
    for bar in bars:
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2, height + 0.5,
                    f'{height:.1f}', ha='center', va='bottom')
    
    # Total cost
    bars = axes[1].bar(comm_df['Method'], comm_df['Total MB'], 
                       color=sns.color_palette('Set2', len(comm_df)))
    axes[1].set_ylabel('Total Communication (MB)')
    axes[1].set_title('Total Communication Cost', fontsize=12, fontweight='bold')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(axis='y', alpha=0.3)
    
    for bar in bars:
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2, height + 5,
                    f'{height:.0f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('outputs/communication_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No communication metrics available for comparison")

## 7. Comprehensive Comparison Table

In [ ]:
# Create comprehensive comparison
comparison_data = []

for name, results in all_results.items():
    test_metrics = results.get('test_metrics', {})
    exp_metrics = results.get('explanation_metrics', {})
    unc_metrics = results.get('uncertainty_metrics', {})
    comm_metrics = results.get('communication_metrics', {})
    
    row = {
        'Method': name,
        'Accuracy': test_metrics.get('accuracy', 0),
        'F1': test_metrics.get('f1', 0),
        'AUC-ROC': test_metrics.get('auc_roc', 0),
        'SHAP Consistency': exp_metrics.get('shap_consistency', np.nan),
        'Coverage': unc_metrics.get('coverage', np.nan),
        'Avg Set Size': unc_metrics.get('avg_set_size', np.nan),
        'Total Comm (MB)': comm_metrics.get('total_mb', np.nan)
    }
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)

print("\n=== Comprehensive Method Comparison ===")
display(comparison_df.style.format({
    'Accuracy': '{:.4f}',
    'F1': '{:.4f}',
    'AUC-ROC': '{:.4f}',
    'SHAP Consistency': '{:.4f}',
    'Coverage': '{:.4f}',
    'Avg Set Size': '{:.2f}',
    'Total Comm (MB)': '{:.1f}'
}, na_rep='N/A').background_gradient(cmap='YlGn', subset=['Accuracy', 'F1', 'AUC-ROC']))

# Save to CSV
comparison_df.to_csv('outputs/method_comparison.csv', index=False)
print("\n✓ Comparison table saved to: outputs/method_comparison.csv")

## 8. Privacy and Interpretability Trade-offs

In [ ]:
# Qualitative comparison
tradeoffs = pd.DataFrame({
    'Method': ['FedNAMs+', 'FedAvg CNN', 'FedAvg + GradCAM', 'Centralized NAM'],
    'Privacy': ['High', 'High', 'High', 'Low'],
    'Interpretability': ['High', 'Low', 'Medium', 'High'],
    'Uncertainty': ['Yes', 'No', 'No', 'Yes'],
    'Communication': ['Medium', 'Medium', 'Medium', 'N/A']
})

print("\n=== Privacy and Interpretability Trade-offs ===")
display(tradeoffs)

# Create radar chart
from math import pi

# Convert qualitative to quantitative
score_map = {'Low': 1, 'Medium': 2, 'High': 3, 'Yes': 3, 'No': 1, 'N/A': 0}
categories = ['Privacy', 'Interpretability', 'Uncertainty']

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

angles = [n / len(categories) * 2 * pi for n in range(len(categories))]
angles += angles[:1]

for idx, row in tradeoffs.iterrows():
    if row['Method'] != 'Centralized NAM':  # Focus on federated methods
        values = [score_map[row[cat]] for cat in categories]
        values += values[:1]
        ax.plot(angles, values, 'o-', linewidth=2, label=row['Method'])
        ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, size=12)
ax.set_ylim(0, 3)
ax.set_yticks([1, 2, 3])
ax.set_yticklabels(['Low', 'Medium', 'High'])
ax.set_title('Privacy-Interpretability-Uncertainty Trade-offs', 
             size=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.savefig('outputs/tradeoffs_radar.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Generate Comparison Report

In [ ]:
# Generate summary report
report = f"""
{'='*70}
FedNAMs+ Baseline Comparison Report
{'='*70}

METHODS COMPARED
{'-'*70}
"""

for name in all_results.keys():
    report += f"- {name}\n"

report += f"""
\nCLASSIFICATION PERFORMANCE
{'-'*70}
"""

for _, row in metrics_df.iterrows():
    report += f"{row['Method']:20s} | Acc: {row['Accuracy']:.4f} | F1: {row['F1-Score']:.4f} | AUC: {row['AUC-ROC']:.4f}\n"

if explanation_data:
    report += f"""
\nEXPLANATION QUALITY
{'-'*70}
"""
    for _, row in explanation_df.iterrows():
        report += f"{row['Method']:20s} | Consistency: {row['SHAP Consistency']:.4f} | Stability: {row['Feature Stability']:.4f}\n"

if uncertainty_data:
    report += f"""
\nUNCERTAINTY QUANTIFICATION
{'-'*70}
"""
    for _, row in uncertainty_df.iterrows():
        report += f"{row['Method']:20s} | Coverage: {row['Coverage']:.4f} | Set Size: {row['Avg Set Size']:.2f}\n"

report += f"""
\nKEY FINDINGS
{'-'*70}
1. FedNAMs+ provides competitive classification performance
2. Built-in interpretability with NAM architecture
3. SHAP-based post-hoc explanations for detailed analysis
4. Conformal prediction for reliable uncertainty quantification
5. Privacy-preserving federated learning

{'='*70}
"""

print(report)

# Save report
with open('outputs/comparison_report.txt', 'w') as f:
    f.write(report)

print("\n✓ Comparison report saved to: outputs/comparison_report.txt")

## Summary

This notebook provided comprehensive comparison of FedNAMs+ against baseline methods:

**Key Advantages of FedNAMs+:**
- Maintains privacy through federated learning
- Provides built-in interpretability via NAM architecture
- Offers post-hoc SHAP explanations for detailed analysis
- Includes uncertainty quantification with conformal prediction
- Achieves competitive classification performance

**Trade-offs:**
- Slightly higher communication cost than standard FedAvg
- More complex architecture than simple CNNs
- Requires calibration set for uncertainty quantification

All comparison visualizations and tables have been saved to the outputs directory.